In [2]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import MinMaxScaler

from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM
from keras.layers import Dropout

In [3]:
# dataset
df = pd.read_csv('ethereum_price_model.csv')
df.head()

,Unnamed: 0,Date,Price,Open,High,Low,Vol.,Change %
0,0,2023-03-08,1553.49,1561.79,1569.70,1548.98,498570,-0.53
1,1,2023-03-07,1561.78,1565.84,1580.95,1536.31,460100,-0.26
2,2,2023-03-06,1565.84,1564.36,1581.13,1555.43,322160,0.09
3,3,2023-03-05,1564.37,1566.73,1587.95,1556.84,313010,-0.15
4,4,2023-03-04,1566.73,1569.45,1577.02,1550.10,247020,-0.14


In [4]:
# training set
df_train = df.iloc[:, 2:3].values

In [5]:
# scaling
sc = MinMaxScaler(feature_range = (0, 1))
df_scaled = sc.fit_transform(df_train)

In [6]:
# creating data with timesteps
X_train = []
y_train = []
for i in range(60, 2555):
    X_train.append(df_scaled[i-60:i, 0])
    y_train.append(df_scaled[i, 0])
X_train, y_train = np.array(X_train), np.array(y_train)

X_train = np.reshape(X_train, (X_train.shape[0], X_train.shape[1], 1))